In [2]:
import gcsfs
import random
from Bio import Phylo
import io

def list_nk_files(bucket_name: str, prefix: str, output_file: str, project: str = None):
    """
    Lists all .nk files within a given GCS directory (recursively) and writes their full GCS paths to a file.
    
    Parameters
    ----------
    bucket_name : str
        The name of the GCS bucket, e.g. 'proevo-ab'.
    prefix : str
        The GCS directory prefix, e.g. 'lineages/clonalTree/output/runs'.
    output_file : str
        Path to the local file where the .nk file paths will be saved.
    project : str, optional
        GCP project ID, if needed.
        
    Returns
    -------
    None
        Writes file paths to the specified output file, one per line.
    """
    fs = gcsfs.GCSFileSystem(project=project)
    
    # Recursively list all files in the prefix
    # fs.walk returns an iterator of tuples (path, directories, files)
    nk_files = []
    for (dir_path, dirs, files) in fs.walk(f"{bucket_name}/{prefix}"):
        for f in files:
            if f.endswith('.nk'):
                nk_files.append(f"{dir_path}/{f}")
    
    # Write all found nk_files to output_file
    with open(output_file, 'w') as out_f:
        for f in nk_files:
            out_f.write(f"{f}\n")


def summarize_tree_sample(n: int, nk_files_list_file: str, project: str = None):
    """
    Samples n .nk files from the full population listed in nk_files_list_file, 
    reads each as a Newick tree, and computes:
    - Average tree height (max root-to-tip distance)
    - Average root-to-tip distance (mean of root-to-leaf distances)
    
    over the sample of n trees.
    
    Parameters
    ----------
    n : int
        Number of .nk files to sample.
    nk_files_list_file : str
        Local path to the file containing a list of .nk file paths (one per line).
    project : str, optional
        GCP project ID, if needed.
    
    Returns
    -------
    dict
        A dictionary containing:
        {
            'sample_size': n,
            'mean_tree_height': float,
            'mean_root_to_tip_distance': float
        }
    """
    fs = gcsfs.GCSFileSystem(project=project)
    
    # Read the list of nk files
    with open(nk_files_list_file, 'r') as f:
        all_nk_files = [line.strip() for line in f if line.strip()]
    
    if len(all_nk_files) < n:
        raise ValueError(f"Requested sample size {n}, but only {len(all_nk_files)} .nk files available.")
    
    # Random sample of n files
    sampled_files = random.sample(all_nk_files, n)

    tree_heights = []
    root_to_tip_means = []

    for fpath in sampled_files:
        # Open the file from GCS
        with fs.open(fpath, 'r') as handle:
            # Parse the tree
            tree = Phylo.read(handle, 'newick')
        
        # Compute distances from the root to all leaves
        leaf_distances = []
        for clade in tree.get_terminals():
            leaf_distances.append(tree.distance(tree.root, clade))
        
        if not leaf_distances:
            # No leaves? Skip or handle differently
            continue
        
        # Tree height: maximum root-to-leaf distance
        height = max(leaf_distances)
        
        # Average root-to-tip distance for this tree
        mean_dist = sum(leaf_distances) / len(leaf_distances)
        
        tree_heights.append(height)
        root_to_tip_means.append(mean_dist)
    
    if not tree_heights or not root_to_tip_means:
        raise ValueError("No valid trees processed. Check that .nk files contain valid trees.")
    
    # Compute overall mean statistics
    mean_tree_height = sum(tree_heights) / len(tree_heights)
    mean_root_to_tip = sum(root_to_tip_means) / len(root_to_tip_means)
    
    return {
        'sample_size': n,
        'mean_tree_height': mean_tree_height,
        'mean_root_to_tip_distance': mean_root_to_tip
    }


# Example usage:


# print(stats)


In [ ]:
list_nk_files('proevo-ab', 'lineages/clonalTree/output/runs', 'nk_file_list.txt', project='your-gcp-project-id')

In [3]:
stats = summarize_tree_sample(1000, 'nk_file_list.txt', project='your-gcp-project-id')
print(stats)

{'sample_size': 1000, 'mean_tree_height': 9.82, 'mean_root_to_tip_distance': 5.0205338237772965}


Tree height is defined as the maximum distance for each tree. Root_to_tip distance is the average of the distances of the whole tree (from one node to another)